## Funktionen

> Potenzielles TODO: Zu jeder hier definierten Funktion schreiben in welchem Notebook sie verwendet werden.

Neben den Importen werden auch drei Funktionen definiert, die über die verwendeten Notebooks hinweg benötigt werden. Für die Verwendung muss ebenfalls dieses Notebook zuerst ausgeführt werden.

> %run Util/01_functions.ipynb

Die erste Funktion, die innerhalb dieses Notebooks definiert wird, befasst sich mit der Umwandlung von einer der drei genutzten Darstellungsweisen in eine andere. Bei diesen Darstellungsweisen handelt es sich um:

> TODO: Für jede Darstellungsweise ein Beispiel aufführen
> TODO: Darstellungsweisen updaten
> TODO: Darstellung in ganzen Sätzen beschreiben

* Die Forsyth-Edward-Notation (FEN). Diese besteht aus insgesamt sechs Teilstrings, die zur Beschreibung einer Schachpartie benötigt werden. Eine FEN besteht aus folgenden Elementen:
    * Die Platzierung der Spielfiguren auf dem Schachbrett.
    * Die Farbe, die gerade am Zug ist.
    * Die Möglichkeiten, noch eine Rochade durchzuführen.
    * Die Felder hinter einem Bauern, der per En Passant geschlagen werden können.
    * Die Anzahl an Halbzügen, seitdem die letzte Figur geschlagen wurde oder ein Bauer vorgestoßen ist.
    * Die Anzahl an durchgeführten Zügen.
* Die gekürzte FEN-Schreibweise. Diese besteht im Gegensatz zur klassischen FEN-Schreibweise nur aus der Platzierung der Spielfiguren und der Farbe, die aktuell am Zug ist.
* Und ein Zwei-Tupel. Dieses Tupel besitzt als ersten Wert einen booleschen Wert, der die aktuelle Farbe darstellt, die am Zug ist. Der zweite Eintrag stellt eine ASCII-Repräsentation eines gesamten Schachbretts dar.

> TODO: Stroetmann hat nicht verstanden welche Parameter mitgegeben werden und welche zurückgegeben wird, muss gegeben sein
> TODO: Stroetmann hat den Zusammenhang zu Absatz vorher nicht verstanden

Die Funktion ``get_board_and_turn`` erstellt nun für eine mitgegebene ``fen`` die gekürzte FEN-Schreibweise und gibt diese zurück.

In [ ]:
def get_board_and_turn(fen):
    split = fen.split()
    short_fen = str(split[0]) + " " + str(split[1])
    return short_fen

> TODO: Darstellungsweise der Parameter und Rückgabewerte evtl. anpassen
> TODO: Begriff "situation" hat ihm im Verlauf der Arbeit nicht verstanden

Als Nächstes wird eine Funktion definiert, die die Suche innerhalb einer ``sequence`` ermöglicht. Diese ``sequence`` ist eine Liste bestehend aus Mengen. In diesen Mengen stehen die Spielsituationen in einer der drei zuvor erwähnten Darstellungsweisen. Bei der ``situation`` muss darauf geachtet werden, dass sie die gleiche Darstellung besitzt wie die Elemente in der ``sequence``. Sofern die Funktion ``find_situation_in_sequence`` die Spielsituation in einer der Mengen findet, wird der Mengenindex der Liste zurückgegeben. Andernfalls lautet der Rückgabewert -1.

In [ ]:
def find_situation_in_sequence(situation, sequence):
    for i in range(len(sequence)):
        if situation in sequence[i]:
            return i
    return -1

Im weiteren Verlauf dieser Studienarbeit werden in dem Notebook ``02_calculation.ipynb`` Ergebnisse berechnet, die für eine spätere Validierung benötigt werden. Da die Ergebnisse eine beachtliche Größe besitzen, werden diese lokal abgespeichert und müssen für eine spätere Verwendung erneut geladen werden. Hiermit wird verhindert, dass der Nutzer vor jeder Verwendung der KI warten muss, bis die Endspielsituation berechnet wurde.
Hierfür wird die Funktion ``load_data`` definiert. Diese nimmt den Dateinamen (``filename``) einer ``.chessTest`` entgegen, die in dem Ordner ``S_n_Results`` gespeichert werden. 
Aus dieser Datei wird die ``piece_list`` sowie eine Liste von Mengen generiert, die Spielsituation in der Integer-Repräsentation (``s_n_sequence_integers``) speichern. Diese werden zurückgegeben.

In [ ]:
def load_data(filename):
    s_n_sequence_integers = []
    with ZipFile("S_n_Results/" + filename + ".chessAI") as zipped:
        with zipped.open(filename + ".pickle") as calculation:
            tmp = pickle.loads(calculation.read())
            piece_list = tmp[0]
            for item in tmp[1:]:
                s_n_sequence_integers.append(item)
    return piece_list, s_n_sequence_integers

> TODO: Kommentare mit Parameter und Rückgabewerte schreiben
> TODO: Konzept mit dem Integer beschreiben

In [ ]:
def to_integer(board, piece_list):
    tmp_list = piece_list.copy()
    piece_map = board.piece_map()
    int_rep = 1 if board.turn else 0

    pawn = chess.Piece.from_symbol("P")
    if pawn in piece_map.values():
        tmp_list.append(pawn)
        # Pawn and Queen will not be on the same board
        # This is in line with the tasks in readme.md
        piece_map[127] = chess.Piece.from_symbol('Q')

    for piece in piece_map:
        index = tmp_list.index(piece_map[piece])
        tmp_list[index] = None
        int_rep |= piece << index * 7 + 1 
        
    return int_rep

> TODO: Kommentare mit Parameter und Rückgabewerte schreiben

In [ ]:
def to_board(int_rep, piece_list):
    tmp_list = piece_list.copy()
    pawn = chess.Piece.from_symbol("P")
    tmp_list.append(pawn)

    n = len(piece_list)
    new_piece_map = {}

    turn = int_rep & 1
    int_rep = int_rep >> 1

    for i in range(len(piece_list)):
        pos = int_rep & 127
        if pos > 63:
            continue
        new_piece_map[pos] = tmp_list[i]
        int_rep = int_rep >> 7

    res = chess.Board(None)
    res.turn = bool(turn)
    res.set_piece_map(new_piece_map)
    return res
